Step1: Replacing the orignial ATP with our ligand HCY, using docking software like gnina.

- Gnina, orginal ligand ATP as autobox, -r newly folded protein, -l cortisol_conf0.sdf 
- Pack them as a new prot-ligand complex, feed to cotimed


Using Cotimedutils.py, processing the probility of predicted AAs, and generate redesigned sequence in a json file, inculde:

- from pocket_residues.json =====> prot_predict_prob.csv
- from prot_predict_prob.csv  =====>  prot_redesign_pocket_sequence
- from prot_redesign_pocket_sequence =====> mutation_comb if needed
- from prot_redesign_pocket_sequence =====> prot_onechain_mut_all (only for symmertic)



In [1]:
import ampal
from helperfunction import cotimedutils
import os
import json
from pathlib import Path


if __name__ == "__main__":

    protein_name = 'AcrR'
    pocket_size = 8
    cutoff = 0.16
    cotimedutils.do_homodimer_pipeline(protein_name, pocket_size, cutoff)



[INFO] Saved filtered data for 'AcrR' → AcrR_predict_prob.csv
Predicted sequence data saved to 'AcrR_redesign_pocket_sequence.json'.
Processing AcrR
Generated 64 mutation combinations.
Mutation combinations: [{'101': 'E', '105': 'L', '108': 'F', '109': 'F', '112': 'R', '114': 'A', '115': 'M', '118': 'I', '119': 'N', '124': 'M', '125': 'Q', '127': 'L', '128': 'F', '131': 'L', '135': 'L', '149': 'E', '151': 'F', '152': 'L', '153': 'H', '155': 'C', '156': 'L', '176': 'I', '297': 'K', '301': 'H', '347': 'I', '351': 'M', '352': 'D', '353': 'P', '63': 'L', '66': 'A', '67': 'L', '70': 'I', '71': 'D', '73': 'I', '74': 'V', '90': 'A', '94': 'L', '95': 'A', '97': 'Q', '98': 'A'}, {'101': 'E', '105': 'L', '108': 'F', '109': 'F', '112': 'R', '114': 'A', '115': 'M', '118': 'I', '119': 'N', '124': 'M', '125': 'Q', '127': 'L', '128': 'F', '131': 'L', '135': 'L', '149': 'E', '151': 'F', '152': 'L', '153': 'H', '155': 'C', '156': 'L', '176': 'I', '297': 'K', '301': 'H', '347': 'I', '351': 'M', '352': '

Double check the sequence if the pipeline works!
should able to generate: 

1. A mutation summary of the redesigned sequences.

In [2]:
# Summary the designed sequences where is the mutations 
# also double check the mutated sequences  against the wildtype sequence

from Bio import AlignIO
from Bio import SeqIO
import json
import os
import pandas as pd


with open ('mutated_homodimer_sequences.json', 'r') as f:
    mutated_sequences = json.load(f)

wildtype_sequence = 'MSSSAPERVRPGRGGILDAATRLFATHGVSGTSLQQIAGAAGITKAAVYHHFPTKEEVVAAVLAPALEAIDALVRTAEAHDEPRTRTEAAIIGLADQAVTHRQRWAVLLQDAAVEEYIRNDPGHDELFTRLRGLLTGPGPDPGTRLQVALFLSGLLGPAQDPSCADIDDDDLRAGIVRAGRLLLLDGAATG'

chainA_length = len('MSSSAPERVRPGRGGILDAATRLFATHGVSGTSLQQIAGAAGITKAAVYHHFPTKEEVVAAVLAPALEAIDALVRTAEAHDEPRTRTEAAIIGLADQAVTHRQRWAVLLQDAAVEEYIRNDPGHDELFTRLRGLLTGPGPDPGTRLQVALFLSGLLGPAQDPSCADIDDDDLRAGIVRAGRLLLLDGAATG')
chainB_length = len('MSSSAPERVRPGRGGILDAATRLFATHGVSGTSLQQIAGAAGITKAAVYHHFPTKEEVVAAVLAPALEAIDALVRTAEAHDEPRTRTEAAIIGLADQAVTHRQRWAVLLQDAAVEEYIRNDPGHDELFTRLRGLLTGPGPDPGTRLQVALFLSGLLGPAQDPSCADIDDDDLRAGIVRAGRLLLLDGAATG')


'''
format:
Sequence_ID {} chainA_mutations | chainB_mutations |

'''
summary_by_seq = {}

total_len = chainA_length + chainB_length
for seq_id, mutated_seq in mutated_sequences.items():
    chain_A_mut = []
    chain_B_mut = []

    L = min(len(wildtype_sequence), len(mutated_seq), total_len)
    for i in range(L):
        if wildtype_sequence[i] != mutated_seq[i]:
            if i < chainA_length:
                chain_A_mut.append(f"{wildtype_sequence[i]}{i+1}{mutated_seq[i]}")
            elif i < total_len:
                posB = i - chainA_length + 1
                chain_B_mut.append(f"{wildtype_sequence[i]}{posB}{mutated_seq[i]}")

    summary_by_seq[seq_id] = {
        "ChainA_Mutations": ";".join(chain_A_mut) if chain_A_mut else "No mutations",
        "ChainB_Mutations": ";".join(chain_B_mut) if chain_B_mut else "No mutations",
    }

# write to csv file
df_summary = pd.DataFrame.from_dict(summary_by_seq, orient='index')
df_summary.to_csv('mutation_summary.csv', index_label='Sequence_ID')

In [2]:
import yaml
from Bio import SeqIO
import os
from helperfunction import boltzutils

# Example usa
if __name__ == "__main__":

    folder_path_current = "/mnt/d/binder_finder_backup/TransF_biosensor/TF_AcrR_redesign/pocket_redesign/cotimed_predict"  # Current working directory, adjust if needed"yaml4boltz"
    folder_path = os.path.join(folder_path_current, "homodimer_fasta_lib")  # Adjust to your folder containing FASTA files
    # for files in folder
    fasta_files = [f for f in os.listdir(folder_path)]

    # create a directory to save the yaml files
    os.makedirs("yaml4boltz_AcrR_EST_nolig", exist_ok=True)
    folder_yaml = "yaml4boltz_AcrR_EST_nolig"  # Directory to save YAML files
    
    for fasta_file in fasta_files:

        # generate a yaml file for boltz2 prediction
        fasta_path = os.path.join(folder_path, fasta_file)
        name_prefix= fasta_file.split("/")[-1].split(".")[0]  # Use the file name as prefix
        output_yaml=f"{name_prefix}.yaml"
        
        # define the ligand smiles, here for AR_MBP it could bind HCY and Maltose
        ligand_smiles= ["C[C@]12CC[C@@H]3c4ccc(cc4CC[C@H]3[C@@H]1CC[C@@H]2O)O","C[C@]12CC[C@@H]3c4ccc(cc4CC[C@H]3[C@@H]1CC[C@@H]2O)O"]  # EST
        
        # Part2: generate yaml for boltz2 prediction without affinity prediction
        boltzutils.fasta_to_boltz_yaml_affinity_off(
            fasta_file=fasta_path,
            output_yaml=output_yaml,
            folder=folder_yaml,
            #ligand_smiles=ligand_smiles,
            #ligand_count=2,
        )
        


Step2: protein structure prediction

- Run boltz
- Also could try chai_1

QC1: Now the pocket redesign part has finished, we could then try to make some figures to check the quality of models.

- Plot1: the quality score of pocket

In [ ]:
from helperfunction import boltzutils
import os

folder_path = "/home/hdwang/sensor_hd/202601/structure_MLT_HCY_nolig/nolig"
boltzutils.prase_boltz_affinity_results(folder_path,prot_name="6OB5")

In [ ]:
from helperfunction import culling
import os

merged_df_process = culling.culling_structure(apo_score_path="/home/hdwang/sensor_hd/202601/structure_MLT_HCY_nolig/nolig/6OB5_confidence_results.csv",
                          holo_score_path="/home/hdwang/sensor_hd/202601/structure_MLT_HCY/6OB5_confidence_results.csv",
                          rosetta_energy_path="/home/hdwang/sensor_hd/202601/structure_MLT_HCY/6OB5_rosetta_energy.csv",
                          quality_threshold=0.75,
                          confidence_threshold=3,
                          )


# merge the pyrosetta scoring results with the 


In [ ]:
culling.plot_quality_distributions  (merged_df_process)

In [ ]:
# extract the pyrosetta data from json and convert to csv
import os
import json
with open('/home/hdwang/sensor_hd/202601/structure_MLT_HCY/6OB5_rosetta_energy.json', 'r') as f:
    data = json.load(f)
import pandas as pd

df  = pd.DataFrame.from_dict(data, orient='index')
df = df.reset_index(names='prot_name')
df.to_csv('6OB5_rosetta_energy.csv', index=False)

In [ ]:

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np


def plot_cotimed_prob(input_csv: str, output_plot: str):
    """
    Plots the predicted probabilities from a CSV file and saves the plot as an image.

    Args:
        input_csv (str): Path to the input CSV file containing predicted probabilities.
        output_plot (str): Path to save the output plot image.
    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(input_csv)

    amino_acids = df.columns[2:]

    # Create a plot for each position, as a homodimer we only plot half of the chain
    for _, row in df.iterrows():

        # only plot half of the wohle row number, as a homodimer we only plot half of the chain
        
        position = f"{row['pdbinfo_chain']}_{int(row['idx'])}"
        probabilities = row[amino_acids]
        
        # Create figure
        plt.figure(figsize=(12, 6))
        bars = plt.bar(amino_acids, probabilities)
        
        # Add value labels
        for bar in bars:
            height = bar.get_height()
            if height > 0.05:  # Only label significant probabilities
                plt.text(bar.get_x() + bar.get_width()/2., height,
                        f'{height:.2f}',
                        ha='center', va='bottom', fontsize=8)
        
        plt.title(f'Amino Acid Probabilities at Position {position}')
        plt.ylabel('Probability')
        plt.ylim(0, 1)
        plt.grid(axis='y', alpha=0.5)
        plt.show()


def plot_cotimed_prob_all(prot_name: str, input_csv: str, output_plot: str, chain_length: int):
    """
    Plots the predicted probabilities for all positions in a grid-like heatmap.

    Args:
        prot_name (str): Name of the protein for the plot title.
        input_csv (str): Path to the input CSV file containing predicted probabilities.
                         Expected columns: 'idx', 'pdbinfo_chain', and then amino acid codes (e.g., 'A', 'C', 'G').
        output_plot (str): Path to save the output plot image.
        chain_length (int): The maximum sequence position (idx) to include in the plot.
    """
    # Read the input CSV file
    df = pd.read_csv(input_csv)

    # Drop the 'pdbinfo_chain' column as it's not needed for the heatmap data
    df = df.drop(columns=['pdbinfo_chain'])

    # Set 'idx' column as the DataFrame index
    df = df.set_index('idx')

    # Filter the DataFrame to only show positions up to chain_length
    # This assumes 'idx' is numerical and represents sequence position.
    df = df[df.index <= chain_length]
    # convert the idx column to int
    df.index = df.index.astype(int)
    # Create the figure and axes for the heatmap
    plt.figure(figsize=(15, 8)) # Increased figure size for better visibility
    ax = sns.heatmap(
        df.T,  # Transpose the DataFrame: Amino acids as rows, positions as columns
        # annot=True, # Uncomment to show values in cells (can be very cluttered for large data)
        fmt=".2f",  # Format annotations to 2 decimal places if annot=True
        cmap="viridis", # Choose a color map (e.g., "viridis", "plasma", "YlGnBu")
        cbar=True,  # Show the color bar
        cbar_kws={'label': 'Probability'} # Label for the color bar
        
    )
    cbar = ax.collections[0].colorbar
    cbar.set_label('Probability', fontsize=18)
    cbar.ax.tick_params(labelsize=16)
    # Customize plot labels and title
    ax.set_xlabel('Sequence Position', fontsize=18)
    ax.set_ylabel('Amino Acid', fontsize=18)
    ax.set_title(f'{prot_name} Amino Acid Probability Heatmap', fontsize=20)


    xtick_indices = np.arange(0, len(df.index), 10)

    # Get the actual sequence position numbers that correspond to these indices.
    # df.index contains the original 'idx' values.
    xtick_labels = df.index[xtick_indices].tolist()

    # Set the x-ticks on the heatmap.
    # We add 0.5 to each index to center the tick label within the heatmap cell.
    ax.set_xticks(xtick_indices + 0.5)

    # Set the labels for the x-ticks, rotate them for readability, and set fontsize.
    # 'ha' (horizontal alignment) is set to 'center' for 90-degree rotation.
    ax.set_xticklabels(xtick_labels, rotation=90, fontsize=16, ha='center')
    # --- Change to make y-ticks larger ---
    ax.tick_params(axis='y', labelsize=16) # Increased labelsize for y-axis
    # Ensure all elements fit within the figure area, especially with rotated labels
    plt.tight_layout()

    # Save the plot to the specified output path
    plt.savefig(output_plot, dpi=300) # Use a high DPI for better image quality

    # Display the plot
    plt.show()







if __name__ == "__main__":
    prot_name = "3cyl_TES2"  # Adjust the protein name as needed
    chain_length = 122
    input_csv = "3cyl_TES2_predict_prob.csv"  # Path to the input CSV file
    output_plot = f"{prot_name}_predicted_probabilities.png"  # Path to save the output plot
    #plot_cotimed_prob(input_csv, output_plot)
    plot_cotimed_prob_all(prot_name,input_csv, output_plot, chain_length=chain_length)


Plot B, the score of affinity and plddt, rmsd
updating 20250624

Section3: Prepare the pdb and  lig for docking. The culling rule is: higher vina score and gnina score



Get protein, clean ligand, and docking

In [ ]:
from helperfunction import boltzutils


# run all files in a folder， get hydrogen added and cleaned boltz pdb files for gnina docking if needed.
boltzutils.do_boltzpdb_clean(
    pdb_boltz_folder="/home/hdwang/sensor_hd/protein_sensor_hd/AR_MBP_demo/timed_design/structure",
    prot_name="6OB5"
)

# parse rmsd if needed 
boltzutils.prase_boltz_pdb_results(
    pdb_boltz_folder="/home/hdwang/sensor_hd/protein_sensor_hd/AR_MBP_demo/timed_design/structure",
    prot_name="6OB5",
    ref_pdb="6OB5.pdb"
)

In [ ]:
import os
import json
import pandas as pd
from multiprocessing import Pool, cpu_count
from pathlib import Path
import eval_mut_comb
import pyrosetta

# Initialize PyRosetta
pyrosetta.init("-mute all -ignore_unrecognized_res -ex1 -ex2aro -use_input_sc")

def process_single_pdb(args):
    """
    Wrapper function for parallel processing with error handling
    
    Args:
        args (tuple): Contains the sub_path and prot_name.
    Returns:

        tuple: (name_prefix, rosetta_energy, error_message)
        name_prefix (str): The name prefix derived from the sub_path.
        rosetta_energy (float or None): The Rosetta energy score, or None if an error occurs.
        error_message (str or None): Error message if an error occurs, otherwise None.

        updating 20250617
    
    """
    sub_path, prot_name = args
    try:
        name_prefix = os.path.basename(sub_path)
        predict_path = os.path.join(sub_path, "predictions")
        affinity_files = os.listdir(predict_path)
        affinity_files_real = os.path.join(predict_path, affinity_files[0])
        
        pdb_path = os.path.join(affinity_files_real, f"{affinity_files[0]}_model_0_cleanH.pdb")
        if not os.path.exists(pdb_path):
            return (name_prefix, None, "No PDB file found")
        
        rosetta_energy = eval_mut_comb.get_interface_scores(pdb_path)
        return (name_prefix, rosetta_energy, None)
    
    except Exception as e:
        return (name_prefix, None, str(e))

def prase_rosetta_score(pdb_boltz_folder: str, prot_name: str):
    """
    Parse Rosetta scores with multiprocessing support
    Args:
        pdb_boltz_folder (str): The path to the folder containing Boltz predictions.
        prot_name (str): The name of the protein, e.g. '1flm'.
    Returns:
        None: Saves the results to a JSON file and a CSV file with errors.

        updating 20250617
    
    """
    subfolders = [f.path for f in os.scandir(pdb_boltz_folder) if f.is_dir()]
    score_all = {}
    error_df = pd.DataFrame(columns=["prot", "error"])
    
    # Prepare tasks for multiprocessing
    tasks = [(sub_path, prot_name) for sub_path in subfolders]
    
    # Use 75% of available CPUs to avoid overloading
    num_processes = max(1, int(cpu_count() * 0.75))
    
    with Pool(processes=num_processes) as pool:
        results = pool.map(process_single_pdb, tasks)
    
    # Process results
    for name_prefix, energy, error in results:
        if error:
            print(f"Error processing {name_prefix}: {error}")
            error_df = pd.concat([
                error_df,
                pd.DataFrame({"prot": [name_prefix], "error": [error]})
            ], ignore_index=True)
        else:
            score_all[name_prefix] = energy
    
    # Save outputs
    error_path = os.path.join(pdb_boltz_folder, f"{prot_name}_pdb_errors.csv")
    error_df.to_csv(error_path, index=False)
    
    json_path = os.path.join(pdb_boltz_folder, f"{prot_name}_rosetta_energy.json")
    with open(json_path, 'w') as f:
        json.dump(score_all, f, indent=4)
    
    print(f"Processed {len(score_all)}/{len(subfolders)} structures successfully")

if __name__ == "__main__":
    pdb_boltz_folder = "/mnt/d/binder_finder_backup/plip_prase/25.04/Cortisol_sensor/1flm/1flm_pdb_boltz"
    prot_name = "1flm"
    prase_rosetta_score(pdb_boltz_folder, prot_name)

Docking with gnina, using below script.
- Put conformer and docking.py in same folder
- Run docking, get the score 
- Run & parse the score

In [ ]:
import os
import time
import subprocess
from multiprocessing import Pool,cpu_count
from pathlib import Path
import pandas as pd

def generate_config(protein_path, autobox_path, conformer_path, output_path_result, output_path_log, config_directory):
    
    config_contents = f"""
    receptor = {protein_path}
    autobox_ligand = {autobox_path}
    ligand = {conformer_path}
    cnn_scoring = rescore
    exhaustiveness = 16
    num_modes = 10
    seed = 42
    cpu = 16
    pose_sort_order = energy
    out = {output_path_result}
    log = {output_path_log}
    """
    repacking_id = Path(protein_path).stem
    config_path = config_directory / f"{repacking_id}.txt"
    config_path.write_text(config_contents.strip())
    return config_path

def run_docking(config_path):
    cmd = f"gnina --config {config_path}"

    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, timeout=300)
        if result.returncode != 0:
            print(f"Error running gnina: {result.stderr.decode()}")
            return False
        else:
            print("Docking completed successfully.")
            return True
    except subprocess.TimeoutExpired:
        print(f"Docking timed out for {config_path}. Skipping.")
        return False

def process_repacking(repack_pdb_path, autobox_path, conformer_path, folder):
    try:
        protein_path = Path(repack_pdb_path)
        repacking_id = protein_path.stem # Extract the base name without extension
        print(f"Processing {repacking_id} with conformer {conformer_path}")
        #autobox_path = f"/mnt/e/autobox/{prot_id}.pdb"
        
        output_path_result = folder / 'result' / (repacking_id + '.pdb')
        output_path_result_folder = output_path_result.parent
        output_path_result_folder.mkdir(parents=True, exist_ok=True)
        
        output_path_log = folder / 'log' / repacking_id
        output_path_log_folder = output_path_log.parent
        output_path_log_folder.mkdir(parents=True, exist_ok=True)
        
        config_directory = folder / 'config'
        config_directory.mkdir(parents=True, exist_ok=True)
        
        config_path = generate_config(str(protein_path), autobox_path, conformer_path, str(output_path_result), str(output_path_log), config_directory)
        return run_docking(config_path)
    except Exception as e:
        print(f"Error processing {repack_pdb_path}: {str(e)}. Skipping.")
        return False


def safe_process_wrapper(args):
    """Wrapper function to handle exceptions in worker processes"""
    try:
        return process_repacking(*args)
    except Exception as e:
        print(f"Unhandled error in worker process: {str(e)}")
        return False

def main(pdb_boltz_folder):
    
    # Lets first set up the directories
    subfolders = [f.path for f in os.scandir(pdb_boltz_folder) if f.is_dir()]
    
    # direct to the predictions folder
    error_df = pd.DataFrame(columns=["prot", "error"])
    score_all = {}  # to collect all scores
    for sub_path in subfolders:
    
        name_prefix = sub_path.split("/")[-1]  # get the name of the subfolder, e.g. 1flm_hcy12_87
        name_prefix = name_prefix.replace('boltz_results', '')  # replace underscores with dashes for compatibility
    # crate a dictionary to store the scores for this protein
        if name_prefix not in score_all:
            score_all[name_prefix] = {}

        # read affinity json file which starts with "affinity_"
        predict_path = os.path.join(sub_path, "predictions")
       
        affinity_files = os.listdir(predict_path)

        affinity_files_real = os.path.join(predict_path, affinity_files[0]) # it should output the folder path containing the pdb files

        # autobox path C = affinity_files_real + 3cyl_TES2_1_model_0_ligandC.pdb
        # receptor = affinity_files_real + 3cyl_TES2_1_model_0_cleanH.pdb
        autobox_path_C = os.path.join(affinity_files_real, f"{name_prefix}_model_0_ligandC.pdb")
        receptor_path = os.path.join(affinity_files_real, f"{name_prefix}_model_0_cleanH.pdb")
        

        start_conformer = 0  # Start from conformer 
        tasks = []
        for i in range(start_conformer, 10):
            print(f"\n=== Processing conformer {i} ===")
            conformer = f'conformers/Testosterone_conf{i}.sdf'
            folder_out = Path(f'repack_conformer{i}')
            folder_out.mkdir(parents=True, exist_ok=True)
            tasks.append((receptor_path, autobox_path_C, conformer, folder_out))

            # Collect all PDB files to process
           
            
            
        # Process tasks in parallel
        num_processes = min(cpu_count()-5, len(tasks))  # Use all available cores
        with Pool(num_processes) as pool:
            results = pool.map(safe_process_wrapper, tasks)
        print(f"Processed {len(results)} tasks with {num_processes} processes.")

if __name__ == "__main__":
    main()


Processing the docking result

In [ ]:
import json
import re
import pandas as pd
import os


# 1.1 analysis log file, and extract the affinity

def analysis_log_file(log_path):

    column_names = ['Mode', 'Affinity (kcal/mol)', 'Intramol (kcal/mol)', 'CNN Pose Score', 'CNN Affinity']
    error_df_row = {column: None for column in column_names}
    
    try:
        with open(log_path, 'r' ) as f:
            data = f.readlines()
        if not data:
            return pd.DataFrame([error_df_row]),f"Empty file: {log_path}"
    except Exception as e:
        return pd.DataFrame([error_df_row]),f"Error reading file: {log_path} : {e}"


    # select the indices to process
    
    #-----------------------------------------------------------------------------------------#
    '''adjust the indices here, for rigid docking, it is 19, for flexible docking, it is 20 '''
    #-----------------------------------------------------------------------------------------#
    indices_to_process = [19] 

    extracted_data = []
    error_message = None

    # iterate over selected indices
    for idx in indices_to_process:
        if idx < len(data) and data[idx].strip():
            values = re.findall(r"[-+]?\d*\.\d+|\d+", data[idx])
            
            values = [float(num) if '.' in num else int(num) for num in values]

            if len(values) == 5: # if the length is 5, then it is the correct format
                extracted_data.append(values)
            else:
                print(f'Error in the data: {values}, expected 5 values, but got {len(values)} at index {idx}')
        else:
            error_message = f"Index {idx} out of range or line is empty in file {log_path}"

    # using a dataframe to store the data
    if not extracted_data:
        return pd.DataFrame([error_df_row]), error_message or f"No data collected from {log_path}"

    return pd.DataFrame(extracted_data, columns=column_names), None


# 1.2 for each folder, process all the log files
def process_folder(folder_Path):
    log_files = os.listdir(folder_Path)
    
    combined_df = []
    error_records = []    

    for log_file in log_files:
        

        file = log_file

        data_df, error_msg = analysis_log_file(os.path.join(folder_Path, file))
        data_df['file'] = file
        combined_df.append(data_df)
       
        if error_msg:  # This file had an issue
            error_records.append({'file': file, 'error': error_msg})
    
    #combine all the dataframes
    all_df = pd.concat(combined_df, ignore_index=True)
    error_df = pd.DataFrame(error_records)

    error_df.to_csv(f'{folder_Path}_errors.csv', index=False)
    all_df.to_csv(f'{folder_Path}.csv', index=False)

# load the data, mainly log files
for i in range(0,9+1):
    folder_path = f'/home/hdwang/protein_sensor_hd/1flm/1flm_1st_docking/repack_conformer{i}/log' # D:\4.Testosterone\1st_redesign_docking\repack_conformer0
    df = process_folder(folder_path)
    print(f'Finished processing {folder_path}')


In [ ]:
def load_data():
    for i in range(0,9+1):
        log_path = f'/home/hdwang/protein_sensor_hd/1flm/1flm_1st_docking/repack_conformer{i}/log.csv'

        df_log = pd.read_csv(log_path)

        df = pd.DataFrame()
        # process each row
        df['affinity'] = df_log['Affinity (kcal/mol)']
        df['gnina_score'] = df_log['CNN Pose Score'] * df_log['CNN Affinity']
        df['file'] = df_log['file'].str.split('_', expand=True)[[0, 1,2]].agg('_'.join, axis=1)


        df.to_csv(f'affinity_gnina_score{i}.csv', index=False)

        print(f'Finished processing {log_path}')



import pandas as pd

'''

dealing with missing value, fill  NaN with the median of each column

'''
# Function to load data and handle missing values
def load_and_process_data(file_paths):
    dfs = []
    for path in file_paths:
        df = pd.read_csv(path)
        # handle missing values
        numeric_columns = df.select_dtypes(include=['number']).columns
        medians = df[numeric_columns].median()
        df[numeric_columns] = df[numeric_columns].fillna(medians)
        dfs.append(df)
    
    # Concatenate all DataFrames into one
    combined_df = pd.concat(dfs, ignore_index=True)
    return combined_df

# Function to calculate aggregation
def aggregate_data(df):
    # Group by 'file' column to sum scores and average affinity
    grouped = df.groupby('file', as_index=False).agg({
        'gnina_score': 'mean',
        'affinity': 'mean'
    })
    return grouped

# Main function to execute the workflow
def main():
    # Define paths to the input files
    file_indices = range(10) # Number of files
    file_paths = [f'affinity_gnina_score{i}.csv' for i in file_indices]
    
    # Process data
    combined_df = load_and_process_data(file_paths)
    
    # Aggregate data
    processed_df = aggregate_data(combined_df)
    
    # Export the processed data to CSV
    processed_df.to_csv('combined_affinity_gnina_score.csv', index=False)
    print("Data has been processed and saved successfully.")

# Execute the main function
if __name__ == "__main__":
    load_data()
    main()
    





Merge vina/gnina/iptm/ddG in one dataframe

In [ ]:
# convert json file to csv fileimport json
import pandas as pd

df = pd.read_json('/home/hdwang/protein_sensor_hd/1flm/1flm_pdb_boltz/1flm_rosetta_energy.json', orient='index')
df['prot_name'] = df.index  # Add index as a new column
df.to_csv('/home/hdwang/protein_sensor_hd/1flm/1flm_pdb_boltz/1flm_rosetta_energy.csv', index=False)

In [ ]:
import os
import pandas as pd


'''

This script is used to parse all results from boltz2, docking, and rosetta energy calculations for a given protein.

'''

def merge_all(path_docking, path_boltz, path_rosetta, output_path):

    # Define the folder 

    # Read the CSV files
    df_docking = pd.read_csv(path_docking)
    df_boltz = pd.read_csv(path_boltz)
    df_rosetta = pd.read_csv(path_rosetta)
    # rename columns file in df_docking to avoid conflicts
    df_docking.rename(columns={'file': 'prot_name'}, inplace=True)
    # do some preprocessing to make sure the name is the same, e.g. in confidence results, the prot_name has a prefix 'boltz_results_', we need to remove it
    df_boltz['prot_name'] = df_boltz['prot_name'].str.replace('boltz_results_', '', regex=False)
    df_rosetta['prot_name'] = df_rosetta['prot_name'].str.replace('boltz_results_', '', regex=False)
    # Merge the DataFrames on 'prot_name'
    df_merged = pd.merge(df_docking, df_boltz, on='prot_name', how='outer', suffixes=('_docking', '_boltz'))# suffixes to avoid column name conflicts
    df_merged = pd.merge(df_merged, df_rosetta, on='prot_name', how='outer', suffixes=('', '_rosetta'))  # Merge with rosetta scores

    # rank by the gnina score (fisrt), and ddG (rosetta energy) second, and iptm third
    
    df_merged['gnina_score_rank'] = df_merged['gnina_score'].rank(ascending=False, method='min')
    df_merged['rosetta_energy_rank'] = df_merged['ddG'].rank(ascending=False, method='min')  # Lower ddG is binding well
    df_merged['iptm_rank'] = df_merged['iptm'].rank(ascending=False, method='min')  # Higher IPTM is better
    
    # write the merged DataFrame to a new CSV file
    
    df_merged.to_csv(output_path, index=False)



path_docking = 'combined_affinity_gnina_score.csv'
path_boltz = '/home/hdwang/protein_sensor_hd/1flm/1flm_pdb_boltz/1flm_confidence_results.csv'
path_rosetta = '/home/hdwang/protein_sensor_hd/1flm/1flm_pdb_boltz/1flm_rosetta_energy.csv'
output_path = '3cyl_all_results.csv'
merge_all(path_docking, path_boltz, path_rosetta, output_path)



Make some cutoff based on gnina/affinity score

In [ ]:

# filter out files with good Vina or gnina scores
import matplotlib.pyplot as plt
import pandas as pd

score_path = '/home/hdwang/protein_sensor_hd/1flm/timed_design/result_csv/1flm_all_results20250624.csv'

df_score = pd.read_csv(score_path)

def filter_vina_scores(df):
    
    # ranking by Vina score
    df = df.sort_values('affinity', ascending=True)
    
    # filter out the top 3% of the Vina scores
    df = df.iloc[:int(len(df)*0.03)]


    return df

def filter_gnina_scores(df):
    
    # ranking by gnina score
    df = df.sort_values('gnina_score', ascending=False)
    
    # filter out the top 3% of the gnina scores
    df = df.iloc[:int(len(df)*0.03)]

    return df

# add


df_high_vina = filter_vina_scores(df_score)
df_high_gnina = filter_gnina_scores(df_score)


df_high_gnina.to_csv('1flm_1st_high_gnina.csv', index=False)
df_high_vina.to_csv('1flm_1st_high_vina.csv', index=False)


# although we could do a merge, but we still want to see how scores are distributed, so we could do a plot- vina-gnina scatter plot
'''-8.43,-8.87,-8.65,-8.8,-8.85,-8.74,-8.56,-8.97,-8.49,-9.38,-8.77,0.28,1flm_FMN_B_123,FMN,0.8333333333333334'''

prot_name = '1flm'
plt.figure(figsize=(10, 6))
plt.scatter(df_score['gnina_score'], df_score['affinity'], alpha=0.5)
plt.title(f'{prot_name} Smina score vs Gnina Score', fontsize=22)
plt.ylabel('Smina score (kcal/mol)', fontsize=20)
plt.xlabel('Gnina Score', fontsize=20)
# set x and y ticks, x starts from 0 , y starts from -12
yticks = [-12, -10, -8, -6, -4, -2, 0]
plt.yticks(yticks, fontsize=20)
xticks = [0, 1, 2, 3 ,4, 5, 6, 7]
plt.xticks(xticks, fontsize=20)
plt.axhline(y=-8.77, color='r', linestyle='--', label='-8.77 kcal/mol WT-score') # we could set a threshold showing the firsting round of docking score or WT smina score
plt.legend(fontsize=20)


plt.grid(True)
plt.show()

PlotC checking the all top candidates

In [ ]:
import pandas as pd
import numpy as np

path2allresult = '/home/hdwang/protein_sensor_hd/1flm/timed_design/result_csv/1flm_all_results20250624.csv'
# Load all results
df_all = pd.read_csv(path2allresult)

# get the top 10% of the affinity and gnina scores, here the gnina is higher then better, affinity is lower then better
df_top_3 = df_all.nlargest(int(len(df_all) * 0.10), 'gnina_score')
df_top_3_affinity = df_all.nsmallest(int(len(df_all) * 0.10), 'affinity')

# meet both conditions
df_top_3_both = df_top_3[df_top_3['prot_name'].isin(df_top_3_affinity['prot_name'])]
df_top_3_both.to_csv('3cyl_1st_top_10_percent.csv', index=False)
# Save the results
print(f"Number of top 10% results: {len(df_top_3_both)}")
print(df_top_3_both)



################# again, for those top candidates, plot the affinity, gnina, ddG
from matplotlib import pyplot as plt

def plot_top_candidates(df, title):
    """
    Plot the top candidates based on affinity and gnina scores.
    
    Args:
        df (pd.DataFrame): DataFrame containing the top candidates.
        title (str): Title for the plot.
    """
    plt.figure(figsize=(12, 6))
    
    # barchart for affinity/ gnina/     ddG

    plt.bar(df_top_3_both['prot_name'], df_top_3_both['affinity'], label='Affinity', alpha=0.6, color='blue')
    plt.bar(df_top_3_both['prot_name'], df_top_3_both['gnina_score'], label='Gnina Score', alpha=0.6, color='orange')
    plt.bar(df_top_3_both['prot_name'], df_top_3_both['ddG'], label='ddG', alpha=0.6, color='green')


    # set y scale to every 5, e.g. 0,5,10 and include the min and max of the scores
    y_min = min(df_top_3_both['affinity'].min(), df_top_3_both['gnina_score'].min(), df_top_3_both['ddG'].min())
    y_max = max(df_top_3_both['affinity'].max(), df_top_3_both['gnina_score'].max(), df_top_3_both['ddG'].max())
   
    yticks = np.arange(
        start = np.floor(y_min / 5) * 5, # round down to the nearest 5
        stop = np.ceil(y_max / 5) * 5 + 1, # round up to the nearest 5
        step = 5
    )
    plt.yticks(yticks)  # Set y-ticks to every 5 units
    plt.ylim(yticks[0], yticks[-1]) # make sure include all
    
    # set y =0 for common reference point
    plt.axhline(0, color='black', linewidth=0.8, linestyle='--')  # Add a horizontal line at y=0 for reference
    plt.title(title, fontsize=16)
    plt.xlabel('Protein Name', fontsize=14)
    plt.ylabel('Scores', fontsize=14)
    plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability


    # add a line for showing wild type ddG
    plt.axhline(y=-79, color='red', linestyle='--', label='Wild Type ddG Threshold (-79.57 kcal/mol)')  # Example threshold line



    plt.legend()
    plt.tight_layout()
    plt.show()      


# Plot the top candidates

plot_top_candidates(df_top_3_both, 'Top 10% Candidates Based on Gnina and Pyrosetta ddG Scores')



In [ ]:
import eval_mut_comb
import os
import pyrosetta

# Initialize PyRosetta
pyrosetta.init("-mute all -ignore_unrecognized_res -ex1 -ex2aro -use_input_sc")
raw_1flm_pdb = '/home/hdwang/protein_sensor_hd/1flm/pdb/1flm.pdb'

ddg = eval_mut_comb.get_interface_scores(raw_1flm_pdb)
print(f"ddG for 1flm: {ddg}")

Section4: prepare the DNA sequences.
- For selected sequences and prase the protein sequences to DNA
- remove the typeII RE sites like BSAI or other sites needed

In [ ]:
from helperfunction import lib_maker
import pandas as pd
import os
#def parse_sequence_file(seq_file_path, df_candidates):

df_top_path = '/home/hdwang/protein_sensor_hd/1flm/timed_design/result_csv/1flm_1st_top_10_percent.csv'
seq_folder_path = '/home/hdwang/protein_sensor_hd/1flm/timed_design/1flm_HCY_seq'
df = pd.read_csv(df_top_path)

for sele_prot in df['prot_name']:
    # get the sequence from the file,
    seq_file = os.path.join(seq_folder_path, sele_prot)

    with open(seq_file, 'r') as f:
        seq = f.readlines()
        seq = [line.strip() for line in seq if line.strip()]  # Remove empty
        print(f"Protein: {sele_prot}, Sequence: {seq[1]}")
        
    # convert seq to DNA sequence, and   
        dna_seq = lib_maker.convert_aa_to_dna(seq[1])

    # add to the dataframe
        df.loc[df['prot_name'] == sele_prot, 'dna_seq'] = dna_seq

        print(f"Converted DNA Sequence for {sele_prot}: {dna_seq}")
    
# Save the updated DataFrame with DNA sequences
#df.to_csv('3cyl_1st_top_10_percent_with_dna_seq.csv', index=False)
print("DNA sequences added to DataFrame and saved to CSV.")